In [23]:
import pandas
import math
from sklearn.metrics import roc_auc_score

data = pandas.read_csv('data-logistic.csv', header=None)
y = data[0]
X = data.drop([0], axis = 1)

def nw1(w1, w2, X, y, k, C):
    l = y.shape[0]
    sum = 0
    for i in range(l):
        sum += y[i] * X[1][i] * (1 - 1 / (1 + math.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))
    return w1 + k / l * sum - k * C * w1

def nw2(w1, w2, X, y, k, C):
    l = y.shape[0]
    sum = 0
    for i in range(l):
        sum += y[i] * X[2][i] * (1 - 1 / (1 + math.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))
    return w2 + k / l * sum - k * C * w2

def lr(X, y, C = 0.0, start = [0, 0], k = 0.1):
    max_iter = 10000
    eps = 1e-5
    w1 = start[0]
    w2 = start[1]
    dw = 10 * eps
    
    i = 0
    while ((i < max_iter) and (dw > eps)):
        i += 1
        w1n = nw1(w1, w2, X, y, k, C)
        w2n = nw2(w1, w2, X, y, k, C)
        dw = ((w1n - w1)**2 + (w2n - w2)**2)**0.5
        w1 = w1n
        w2 = w2n
    
    print('W1 =', "%.3f" % w1, ', W2 =', "%.3f" % w2, ', Number of iterations:', i, ', k = ', k, ', C = ', C)
    return [w1, w2]

def scores(w1, w2, X):
    score = list()
    for i in range(X.shape[0]):
        score.append(1 / (1 + math.exp(- w1 * X[1][i] - w2 * X[2][i])))
    return score
        

print('Standard')
[w1, w2] = lr(X, y)
print('Regularized')
[w1r, w2r] = lr(X, y, C = 10)
print('Big step')
lr(X, y, k = 1)
print('Small step')
lr(X, y, k = 0.001)

y_s = pandas.Series(scores(w1, w2, X))
y_r = pandas.Series(scores(w1r, w2r, X))
score_st = roc_auc_score(y, y_s)
score_reg = roc_auc_score(y, y_r)
print('Regression score:', "%.3f" % score_st, 'Regularized regression score:', "%.3f" % score_reg)

Standard
W1 = 0.288 , W2 = 0.092 , Number of iterations: 244 , k =  0.1 , C =  0.0
Regularized
W1 = 0.029 , W2 = 0.025 , Number of iterations: 8 , k =  0.1 , C =  10
Big step
W1 = 0.288 , W2 = 0.092 , Number of iterations: 32 , k =  1 , C =  0.0
Small step
W1 = 0.257 , W2 = 0.120 , Number of iterations: 5126 , k =  0.001 , C =  0.0
Regression score: 0.927 Regularized regression score: 0.936
